# 0. Python and OpenCV version

In [ ]:
import cv2
from platform import python_version
print("Python version : " + python_version())
print("cv2 version : " + cv2.__version__)
# Python version : 3.8.2
# cv2 version : 4.4.0

# 1. Extract Information about a video

In [ ]:
import cv2

# write a vide path which you want to get information
video_path = "/Users/okumurayuki/Desktop/videos_0920/19500Hz.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("width:{}, height:{}, count:{}, fps:{}".format(width,height,count,fps))

# 2. Preprocessing (divide each video frame to an image and save)
*** If you have done this process in Task6(b), you can skip here***

In [ ]:
import os
import cv2
from glob import glob

#write path where your videos locate
dir_name = "/Users/okumurayuki/Desktop/sample"

for video_path in glob(dir_name + '/*.mp4'):

    cap = cv2.VideoCapture(video_path)
    
    #make a folder in the same folder with  its video name 
    #video_path[:-4]  is a folder name where devided frames are saved
    folder_path = video_path[:-4]
    if not os.path.isdir(folder_path):
            os.makedirs(folder_path)

    num = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # save image from frame000
            cv2.imwrite(folder_path +"/frame{:0=3}".format(num)+".jpg",frame)       
            num += 1
        else:
            break

    cap.release()

# 3. Evaluate each frame by Laplacian, Sobel and FFT and save results as image and csv

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 

#write a folder path where videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

for folder_path in glob(dir_name + '/**/'):
    video_path = folder_path[:-1] + ".mp4"
    cap = cv2.VideoCapture(video_path)
    count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    #define list for each evaluation method
    lap_var = []
    sobel_var = []
    fft_mean = []


    for i in range(int(count)):
        image_path = folder_path + "frame{:0=3}".format(i)+".jpg"
        image = cv2.imread(image_path)
        
        #change each image to gray scale
        gray_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        
        #Laplacian
        laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
        lap_var.append(laplacian.var())
        
        #Sobel
        #edge detection with x,y direction
        gray_sobelx = cv2.Sobel(gray_image,cv2.CV_32F,1,0)
        gray_sobely = cv2.Sobel(gray_image,cv2.CV_32F,0,1)
    
        #8-bit unsigned integer conversion
        gray_abs_sobelx = cv2.convertScaleAbs(gray_sobelx) 
        gray_abs_sobely = cv2.convertScaleAbs(gray_sobely)
        #Weighted sum
        gray_sobel_edge = cv2.addWeighted(gray_abs_sobelx,0.5,gray_abs_sobely,0.5,0) 
        sobel_var.append(gray_sobel_edge .var())
        
        #FFT
        # apply fft to a gray image
        fimg = np.fft.fft2(gray_image)
        # swap 1st and 3rd quadrants, 2nd and 4th quadrants to relovate row frequency part to the center of an image
        fimg =  np.fft.fftshift(fimg)
        # calculate power spectrum
        mag = 20*np.log(np.abs(fimg))
            
        # Caluculare mean of mag value from center 25% of area
        #define 50% of area of  image 
        height, width= image.shape[:2]
        lower_width = int(0.25 * width)
        upper_width = int(0.75 * width)
        lower_height = int(0.25 * height)
        upper_height = int(0.75 * height)

        sum_center_mag = 0
        counter = 0
        for j in range(lower_height,upper_height):
            for k in range(lower_width,upper_width):
                sum_center_mag += mag[j][k]
                counter += 1
        fft_mean.append(sum_center_mag/counter)
        
    # result visualization and save results as image and csv
    x = list(range(int(count)))
    y_lap = lap_var
    y_sobel = sobel_var
    y_fft = fft_mean

    fig, (ax1,ax2, ax3) = plt.subplots(ncols=3, figsize=(21,5))
    
    #define figure tittle which is same with the video title
    folder_path = folder_path[:-1]
    fig.suptitle(os.path.basename(folder_path))
    
    #Laplacian
    ax1.plot(x,y_lap)
    ax1.grid(True) 
    ax1.set_xlim(left=x[0], right=x[-1])  
    ax1.set_xlabel('Video frame') 
    ax1.set_ylabel('Variance')
    ax1.set_title('Laplacian')  
    
    #Sobel
    ax2.plot(x,y_sobel)
    ax2.grid(True) 
    ax2.set_xlim(left=x[0], right=x[-1])  
    ax2.set_xlabel('Video frame') 
    ax2.set_ylabel('Variance')  
    ax2.set_title('Sobel')  

    #FFT
    ax3.plot(x,y_fft)
    ax3.grid(True) 
    ax3.set_xlim(left=x[0], right=x[-1])  
    ax3.set_xlabel('Video frame') 
    ax3.set_ylabel('Mean') 
    ax3.set_title('FFT')  

    # save result as jpg and csv
    save_folder = dir_name + '/Blurriness/'
    if not os.path.isdir(save_folder):
            os.makedirs(save_folder)
            
    df = pd.DataFrame({'frame': x, 'Laplacian': y_lap, 'Sobel': y_sobel,'FFT': y_fft})
    blankIndex = [''] * len(df)
    df.index = blankIndex
    df.to_csv(save_folder + os.path.basename(folder_path) + '.csv')
    fig.savefig(save_folder + os.path.basename(folder_path) + '.jpg')  

# 4.1 make a summary of 3 by caluculating "mean" after 100th frame

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 

#write a folder path where videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

#define lists for summary
Frequency = []
Laplacian = []
Sobel = []
FFT = []

for folder_path in glob(dir_name + '/Blurriness' + '/*.csv'):
    df = pd.read_csv(folder_path)
    #define lists for caluculating mean of laplaican, sobel and FFT
    laplacian_mean = []
    sobel_mean = []
    fft_mean = []
    
    #delete first 100 frame results
    for i in range(len(df['Laplacian'])):
        if i >= 100:
            laplacian_mean.append(df['Laplacian'][i])
            sobel_mean.append(df['Sobel'][i])
            fft_mean.append(df['FFT'][i])              
    
    Frequency.append(int(os.path.basename(folder_path[:-6])))
    Laplacian.append(sum(laplacian_mean)/len(laplacian_mean))
    Sobel.append(sum(sobel_mean)/len(sobel_mean))
    FFT.append(sum(fft_mean)/len(fft_mean))

#sort because the list of Frequency is not ordered 
Frequency_sorted = sorted(Frequency)
Laplacian_sorted = []
Sobel_sorted = []
FFT_sorted = []

for i in range(len(Frequency_sorted)):
    for j in range(len(Frequency_sorted)):
        if Frequency_sorted[i] == Frequency[j]:
            Laplacian_sorted.append(Laplacian[j])
            Sobel_sorted.append(Sobel[j])
            FFT_sorted.append(FFT[j])            
            
#show result with graph
fig, (ax1,ax2, ax3) = plt.subplots(ncols=3, figsize=(21,5))
fig.suptitle("Summary")
   
#Laplacian
ax1.plot(Frequency_sorted,Laplacian_sorted)
ax1.grid(True) 
ax1.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])    
ax1.set_xlabel('Frequency [Hz]') 
ax1.set_ylabel('Mean')
ax1.set_title("Laplacian")  
    
#Sobel
ax2.plot(Frequency_sorted,Sobel_sorted)
ax2.grid(True) 
ax2.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])  
ax2.set_xlabel('Frequency [Hz]') 
ax2.set_ylabel('Mean')  
ax2.set_title("Sobel")  

#FFT
ax3.plot(Frequency_sorted,FFT_sorted) 
ax3.grid(True) 
ax3.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])  
ax3.set_xlabel('Frequency [Hz]') 
ax3.set_ylabel('Mean') 
ax3.set_title("FFT")  

# save result as jpg and csv
summary_folder = dir_name + "/Blurriness/Summary/"
if not os.path.isdir(summary_folder):
            os.makedirs(summary_folder)
df = pd.DataFrame({'frame': Frequency_sorted, 'Laplacian': Laplacian_sorted, 'Sobel': Sobel_sorted,'FFT':FFT_sorted})
blankIndex = [''] * len(df)
df.index = blankIndex
df.to_csv(summary_folder + '/Summary_mean.csv')
fig.savefig(summary_folder + "/Summary_mean.jpg")  

# 4.2 make a summary of 3 by caluculating "mode" after 100th frame

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 
import statistics
import math

dir_name = "/Users/okumurayuki/Desktop/videos_0920"

Frequency = []
Laplacian = []
Sobel = []
FFT = []


for folder_path in glob(dir_name + '/Blurriness' + '/*.csv'):
    
    laplacian_round = []
    sobel_round =[]
    fft_round = []
    
    df = pd.read_csv(folder_path)
    
    for i in df['Laplacian'][100:]:
        laplacian_round.append(math.floor(i*10)/10)
    for i in df['Sobel'][100:]:
        sobel_round.append(math.floor(i*10)/10)
    for i in df['FFT'][100:]:
        fft_round.append(math.floor(i*10)/10)
            
    Frequency.append(int(os.path.basename(folder_path[:-6])))
    Laplacian.append(statistics.mode(laplacian_round))   
    Sobel.append(statistics.mode(sobel_round))   
    FFT.append(statistics.mode(fft_round))    
    
#sort because the list of Frequency is not ordered 
Frequency_sorted = sorted(Frequency)
Laplacian_sorted = []
Sobel_sorted = []
FFT_sorted = []

for i in range(len(Frequency_sorted)):
    for j in range(len(Frequency_sorted)):
        if Frequency_sorted[i] == Frequency[j]:
            Laplacian_sorted.append(Laplacian[j])
            Sobel_sorted.append(Sobel[j])
            FFT_sorted.append(FFT[j])
            
#show result with graph
fig, (ax1,ax2, ax3) = plt.subplots(ncols=3, figsize=(21,5))
fig.suptitle("Summary")
    
#Laplacian
ax1.plot(Frequency_sorted,Laplacian_sorted)
ax1.grid(True) 
ax1.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])   
ax1.set_xlabel('Frequency [Hz]') 
ax1.set_ylabel('Mode')
ax1.set_title("Laplacian")  
    
#Sobel
ax2.plot(Frequency_sorted,Sobel_sorted)
ax2.grid(True) 
ax2.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])  
ax2.set_xlabel('Frequency [Hz]') 
ax2.set_ylabel('Mode')  
ax2.set_title("Sobel")  

#FFT
ax3.plot(Frequency_sorted,FFT_sorted) 
ax3.grid(True) 
ax3.set_xlim(left= Frequency_sorted[0], right=Frequency_sorted[-1])   
ax3.set_xlabel('Frequency [Hz]') 
ax3.set_ylabel('Mode') 
ax3.set_title("FFT")  

# save result as jpg and csv
summary_folder = dir_name + "/Blurriness/Summary/"
if not os.path.isdir(summary_folder):
            os.makedirs(summary_folder)
df = pd.DataFrame({'frame': Frequency_sorted, 'Laplacian': Laplacian_sorted, 'Sobel': Sobel_sorted,'FFT':FFT_sorted})
blankIndex = [''] * len(df)
df.index = blankIndex
df.to_csv(summary_folder + '/Summary_mode.csv')
fig.savefig(summary_folder + "/Summary_mode.jpg") 